# load pennai ML options

In [ ]:
import json                                                                          
import numpy as np                                                                   
from json import JSONDecoder                                                         
from functools import partial                                                        
                                                                         
alldata= json.load(open('../../../dockers/dbmongo/files/projects.json'),encoding="utf-8")

# path = 'projects_newlines.json'                                
# data,alldata=[],[]                                                                   
# with open(path) as f:                                                                
#     for line in f:                                                                   
#         alldata.append(json.loads(line))                                             
#     # for data in json_parse(f):                                                     
#     #     print(data)                                                                
#     #     alldata.append(data)                                                       
pennai_ml_opts = {}                                                                         
import pdb                                                                           
from sklearn.model_selection import ParameterGrid                                    
                                                                                     
original_pennai_ml_opts = {}
for d in alldata:                                                                    
    print('ml:',d['name'])                                                           
                                                                                     
    params = d['schema'].keys()                                                      
    choices = {}                                                                     
    for p in params:                               
        if 'values' in d['schema'][p]['ui']:
            choices[p] = d['schema'][p]['ui']['values']                                 
        else:
            choices[p] = d['schema'][p]['ui']['choices']                                 
        print('param:',p,', options:',choices[p])
    # add None to DecisionTreeClassifier's max_depth option.
    if d['name'] == 'DecisionTreeClassifier':
        choices['max_depth'].append(None)
#     pdb.set_trace()
    original_pennai_ml_opts[d['name']] = choices.keys()
    param_grid = list(ParameterGrid(choices))                                        
#     print('param_grid contains',len(param_grid),'options')                           
    pennai_ml_opts[d['name']] = param_grid                                                  
print('pennai_ml_opts:',len(pennai_ml_opts))      
for key in pennai_ml_opts.keys():
    print(key,'options:',len(pennai_ml_opts[key]))

## method to extend ML parameter options with default parameters if not present

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
default_params = {}
default_params['DecisionTreeClassifier'] = DecisionTreeClassifier().get_params()
default_params['RandomForestClassifier'] = RandomForestClassifier().get_params()
default_params['ExtraTreesClassifier'] = ExtraTreesClassifier().get_params()
default_params['GradientBoostingClassifier'] = GradientBoostingClassifier().get_params()
default_params['BernoulliNB'] = BernoulliNB().get_params()
default_params['GaussianNB'] = GaussianNB().get_params()
default_params['MultinomialNB'] = MultinomialNB().get_params()
default_params['KNeighborsClassifier'] = KNeighborsClassifier().get_params()
default_params['LinearSVC'] = LinearSVC().get_params()
default_params['SVC'] = SVC().get_params()
default_params['LogisticRegression'] = LogisticRegression().get_params()

def extend_params(ml,params):
    missing_params = [(k,v) for k,v in default_params[ml].items() if k not in params]
    for k,v in missing_params:
        params[k] = v
    return params

##
# function to lower case strings
def fix_params(params):
    for k,v in params.items():
        if type(v) is str:
            if v == 'true':
                params[k] = True
            elif v == 'false':
                params[k] = False
            elif v == 'None':
                params[k] == None
            else:
                params[k] = v.lower() 
    return params

In [ ]:
pennai_ml_opts_ext = {}
for k,v in pennai_ml_opts.items():
    pennai_ml_opts_ext[k] = []
    for params in v:
        pennai_ml_opts_ext[k].append(extend_params(k,params))

# load pmlb results
filter out classifiers not in PennAI

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('../../../mock_experiment/sklearn-benchmark5-data-edited.tsv.gz', sep='\t').fillna('')
print(data.head())
data['accuracy'] = data['accuracy'].apply(lambda x: round(x, 3))
print('loaded ',data['dataset'].unique().shape[0],'datasets and ', data['classifier'].unique().shape[0],'classifiers')
# subset data to classifiers used in PennAI
pennai_classifiers = ['LogisticRegression', 'RandomForestClassifier', 'SVC', 
                      'KNeighborsClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier']
mask = np.array([c in pennai_classifiers for c in data['classifier'].values])
data = data.loc[mask,:]
print('datasets (',len(data['dataset'].unique()),')')
print('classifiers (',len(data['classifier'].unique()),'):',data['classifier'].unique())
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].unique()),'):',df_g['parameters'].unique()[:5])


In [ ]:
data.loc[data.classifier=='GradientBoostingClassifier']['parameters'].unique()

# convert from sklearn-style parameter formatting to dictionary

In [ ]:
def fix_params(p):
    fixed_params = {k:v for k,v in [tuple(ps.split('=')) for ps in filter(None, p.split(','))]}
    for k,v in fixed_params.items():
        try:
            fixed_params[k] = int(v)
        except ValueError:
            try:     
                fixed_params[k] = float(v)
            except ValueError:
                if fixed_params[k] == 'None':
                    fixed_params[k] = None 
                else:
                    fixed_params[k] = str(v).lower()
                pass
    return fixed_params
# datanew = data.copy()
data_formatted = data.copy()
data_formatted['parameters'] = data_formatted['parameters'].apply(lambda x: fix_params(x))

data_formatted.head()

## add default parameters to parameter dictionaries

In [ ]:
def df_extend_params(x):
#     print(x)
    x.parameters = extend_params(x.classifier,x.parameters)
    return x
data_formatted_ext = data_formatted.copy()
data_formatted_ext = data_formatted_ext.apply(lambda x: df_extend_params(x),axis=1)

In [ ]:
data_formatted_ext.head()

# filter out any parameter combinations that PennAI can't recommend

In [ ]:
def valid_param_combo(ml, params):
    if params in pennai_ml_opts_ext[ml]:
        return True
    else:
        return False
print('unfiltered:',len(data_formatted_ext),'results with',
     len(data_formatted_ext.parameters.apply(str).unique()),'unique parameters')
data_formatted_ext_filtered = data_formatted_ext#.loc[data_formatted_ext.classifier=='SVC']
print('calculating mask...')
mask = []
from tqdm import tqdm
for _, row in tqdm(data_formatted_ext_filtered.iterrows()):
    mask.append(valid_param_combo(row['classifier'],row['parameters']) )
print('done with mask')
# mask = mask | data_formatted_ext_filtered.classifier=='LogisticRegression'
data_formatted_ext_filtered = data_formatted_ext_filtered.loc[mask]
print('filtered data has',len(data_formatted_ext_filtered),'results with',
     len(data_formatted_ext_filtered.parameters.apply(str).unique()),'unique parameters')
data_formatted_ext_filtered.head()
# data_filtered = data.loc[lambda x: valid_param_combo(i['classifier'],i['parameters']) for i in x]


In [ ]:
data_formatted_ext_filtered.classifier.unique()

In [ ]:
print('pennai_ml_opts:')
print(pennai_ml_opts_ext['GradientBoostingClassifier'][0].keys(),len(pennai_ml_opts_ext['GradientBoostingClassifier'][0].keys()))
print('data opts:')
print(data_formatted_ext_filtered.loc[data_formatted_ext_filtered.classifier=='GradientBoostingClassifier'].parameters.values[0].keys())
len(data_formatted_ext_filtered.loc[data_formatted_ext_filtered.classifier=='GradientBoostingClassifier'].parameters.values[0].keys())

In [ ]:
for clf, dfg in data_formatted_ext_filtered.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = data_formatted_ext_filtered['classifier']+'|'+data_formatted_ext_filtered['parameters'].apply(str)
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

# re-prune parameters from knowledgebase that were added as defaults (not in PennAI)

In [ ]:
def prune_params(x):
#     pdb.set_trace()
    ml = x.classifier
    params = x.parameters
    param_opts = original_pennai_ml_opts[ml]
    x.parameters = {k:v for k,v in params.items() if k in param_opts}
    return x 
data_formatted_ext_filtered_pruned = data_formatted_ext_filtered.copy()
data_formatted_ext_filtered_pruned = data_formatted_ext_filtered_pruned.apply(
    lambda x: prune_params(x),axis=1)
    


In [ ]:
for clf, dfg in data_formatted_ext_filtered_pruned.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = (data_formatted_ext_filtered_pruned['classifier']+'|'
              +data_formatted_ext_filtered_pruned['parameters'].apply(str))
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

# subsample the SVC parameter results because they are so imbalanced

In [ ]:
data_formatted_ext_filtered_pruned_lessSVC = data_formatted_ext_filtered_pruned.copy()
dfxfpl = data_formatted_ext_filtered_pruned_lessSVC
C  = [0.001, 0.1, 1, 10]
gamma = [0.01, 1.0, 100]
coef0 = [0.0,1.0,10]

svc = dfxfpl['classifier']=='SVC'
mask = [False for g in svc]
for n in C:
    mask = mask | (svc & 
                   np.array(
                       [p['C'] == n if 'C' in p.keys() else False for p in dfxfpl['parameters'].values]))
mask = (mask | ~svc)
dfxfpl = dfxfpl.loc[mask,:]

svc = dfxfpl['classifier']=='SVC'
mask = [False for g in svc]
for n in gamma:
    mask = mask | (svc &  np.array(
                   [p['gamma'] == n if 'gamma' in p.keys() else False for p in dfxfpl['parameters'].values]))
mask = (mask | ~svc)
dfxfpl = dfxfpl.loc[mask,:]

svc = dfxfpl['classifier']=='SVC'
mask = [False for g in svc]
for n in coef0:
    mask = mask | (svc & np.array(
        [p['coef0'] == n if 'coef0' in p.keys() else False for p in dfxfpl['parameters'].values]))
mask = (mask | ~svc)
dfxfpl = dfxfpl.loc[mask,:]

for clf, dfg in dfxfpl.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = (dfxfpl['classifier']+'|'
              +dfxfpl['parameters'].apply(str))
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

In [ ]:
# Remove big files since their metafeatures take forever to load
# UPDATE: timing isn't an issue anymore since metafeatures for knowledgebases are provided by file. 
# no need to filter these out now.
# data_formatted_ext_filtered_pruned_small = data_formatted_ext_filtered_pruned.copy()
# #datasets to remove:
# big_datasets = ['poker', 'kddcup', 'sleep', 'fars', 'mnist'] 
# # , 'connect-4', 'shuttle', 'adult', 'krkopt', 
# #                 'letter', 'magic', 'nursery', 'pendigits', 'coil2000', 'agaricus-lepiota','optdigits']
# mask = np.array([d not in big_datasets for d in data_formatted_ext_filtered_pruned_small['dataset'].values])
# data_formatted_ext_filtered_pruned_small  = data_formatted_ext_filtered_pruned_small.loc[mask,:]
# print(len(data_formatted_ext_filtered_pruned_small['dataset'].unique()),'datasets left')

# write modified data to file

In [ ]:
print(len(dfxfpl),'total results')
dfxfpl.rename(columns={'classifier':'algorithm'},inplace=True)
dfxfpl.to_csv('sklearn-benchmark5-data-knowledgebase.tsv.gz',
                                                compression='gzip',index=False,sep='\t')

In [ ]:
# make a small knowledge base for testing
test_data = dfxfpl.sample(frac=0.1)
test_data.to_csv('sklearn-benchmark5-data-knowledgebase-small.tsv.gz',
                                                compression='gzip',index=False,sep='\t')